## 기본 파이썬

In [1]:
import glob
import os
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

In [2]:
input_folder = './data/'
output_file = './output_files/05-3 output.xls'

In [3]:
output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('sums_and_averages')

In [4]:
all_data = []
sales_column_index = 3

header = ['workbook', 'worksheet', 'worksheet_total', 'worksheet_average', 'workbook_total', 'workbook_average']

all_data.append(header)

In [5]:
for input_file in glob.glob(os.path.join(input_folder, '*.xls*')):
    with open_workbook(input_file) as workbook:
        list_of_totals = []
        list_of_numbers = []
        workbook_output = []
        
        for worksheet in workbook.sheets():
            total_sales = 0
            number_of_sales = 0
            worksheet_list = []
            worksheet_list.append(os.path.basename(input_file))
            worksheet_list.append(worksheet.name)
            for row_index in range(1, worksheet.nrows):
                try:
                    total_sales += float(str(worksheet.cell_value(row_index, sales_column_index)).strip('$').replace(',', ''))
                    number_of_sales += 1.
                except:
                    total_sales += 0.
                    number_of_sales += 0.
            average_sales = '%.2f' % (total_sales / number_of_sales)
            worksheet_list.append(total_sales)
            worksheet_list.append(float(average_sales))
            list_of_totals.append(total_sales)
            list_of_numbers.append(float(number_of_sales))
            workbook_output.append(worksheet_list)
        workbook_total = sum(list_of_totals)
        workbook_average = sum(list_of_totals) / sum(list_of_numbers)
        for list_element in workbook_output:
            list_element.append(workbook_total)
            list_element.append(workbook_average)
        all_data.extend(workbook_output)
for list_index, output_list in enumerate(all_data):
    for element_index, element in enumerate(output_list):
        output_worksheet.write(list_index, element_index, element)
output_workbook.save(output_file)

## 팬더스

In [6]:
import pandas as pd

output_file = './output_files/05-3 pandas_output.xls'

In [7]:
all_workbooks = glob.glob(os.path.join(input_folder, '*.xls*'))

data_frames = []

In [10]:
for workbook in all_workbooks:
    all_worksheets = pd.read_excel(workbook, sheet_name=None, index_col=None)
    workbook_total_sales = []
    workbook_number_of_sales = []
    worksheet_data_frames = []
    worksheets_data_frame = None
    workbook_data_frame = None
    for worksheet_name, data in all_worksheets.items():
        total_sales = pd.DataFrame([float(str(value).strip('$').replace(',','')) for value in data.loc[:, 'Sale Amount']]).sum()
        number_of_sales = len(data.loc[:, 'Sale Amount'])
        average_sales = pd.DataFrame(total_sales / number_of_sales)
        
        workbook_total_sales.append(total_sales)
        workbook_number_of_sales.append(number_of_sales)
        
        data = {
            'workbook': os.path.basename(workbook),
            'worksheet': worksheet_name,
            'worksheet_total': total_sales,
            'worksheet_average': average_sales
        }
        
        worksheet_data_frames.append(pd.DataFrame([data], columns=['workbook', 'worksheet', 'worksheet_total', 'worksheet_average']))
    worksheets_data_frame = pd.concat(worksheet_data_frames, axis=0, ignore_index=True)
    workbook_total = pd.DataFrame(workbook_total_sales).sum()
    workbook_total_number_of_sales = pd.DataFrame(workbook_number_of_sales).sum()
    workbook_average = pd.DataFrame(workbook_total / workbook_total_number_of_sales)
    workbook_stats = {
        'workbook': os.path.basename(workbook),
        'workbook_total': workbook_total,
        'workbook_average': workbook_average
    }
    workbook_stats = pd.DataFrame([workbook_stats], columns=['workbook', 'workbook_total', 'workbook_average'])
    workbook_data_frame = pd.merge(worksheets_data_frame, workbook_stats, on='workbook', how='left')
    data_frames.append(workbook_data_frame)

all_data_concetenated = pd.concat(data_frames, axis=0, ignore_index=True)

In [11]:
writer = pd.ExcelWriter(output_file)

all_data_concetenated.to_excel(writer, sheet_name='all_data_all_workbooks', index=False)

writer.save()